In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!nvidia-smi

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import confusion_matrix, f1_score
import pandas as pd
import seaborn as sn

In [0]:
!mkdir './data/'
!unzip "/content/drive/My Drive/DeepAssignment/Assignment05/Assignment 5 Dataset.zip" -d "./data/"

In [0]:
data_dir = './data/Assignment 5 Dataset/'

In [0]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
val_data = datasets.ImageFolder(data_dir + '/validation', transform=test_transforms)


trainloader      = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
testloader       = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True)
validationloader = torch.utils.data.DataLoader(val_data, batch_size=8, shuffle=True)

print("Classes: ")
class_names = train_data.classes
print(class_names)

In [0]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

inputs, classes = next(iter(trainloader))
show_databatch(inputs, classes)

## Task1 (i): VGG16

In [0]:
# doing
model = models.vgg16(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=25088, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.classifier = nn.Sequential(*fc_layers)
for param in model.features.parameters():
    param.requires_grad = False
name = 'vgg16'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task1/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

## Task1 (ii): ResNet18

In [0]:
model = models.resnet18(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=512, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(*fc_layers)
name = 'resnet18'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task1/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

## Task2 (i)- a VGG16 (Conv1, Conv2, Conv3 are frozen)

In [0]:
model = models.vgg16(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=25088, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
models.classifier = nn.Sequential(*fc_layers)
for i,param in enumerate(model.features[:17].parameters()): # freezing 7 conv layers, and remaining 6 are unfrozen
    param.requires_grad = False
name = 'vgg16'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2a/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

## Task2 (i)- b VGG16 (Conv1, Conv2 are frozen)

In [0]:
model = models.vgg16(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=25088, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
models.classifier = nn.Sequential(*fc_layers)
for param in model.features[:10].parameters(): # freezing 4 conv layers, and remaining 9 are unfrozen
    param.requires_grad = False
name = 'vgg16'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2b/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

## Task2 (i)- c VGG16 (nothing is frozen)

In [0]:
model = models.vgg16(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=25088, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.classifier = nn.Sequential(*fc_layers)
name = 'vgg16'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2c/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

## Task2 (ii)- a resnet18 (Layer1, Layer2, Layer3 are frozen)

In [0]:
model = models.resnet18(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=512, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.fc = nn.Sequential(*fc_layers)
for i, child in enumerate(model.children()):
    if i < 7:
        for param in child.parameters():
            param.requires_grad = False
name = 'resnet18'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2a/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

## Task2 (ii)- b resnet18 (layer1 is frozen)

In [0]:
model = models.resnet18(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=512, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.fc = nn.Sequential(*fc_layers)
for i, child in enumerate(model.children()):
    if i < 5:
        for param in child.parameters():
            param.requires_grad = False
name = 'resnet18'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2b/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

## Task2 (ii)- c (nothing is frozen)

In [0]:
model = models.resnet18(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=512, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.fc = nn.Sequential(*fc_layers)
name = 'resnet18'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2c/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

## Training

In [0]:
Epochs = 25
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [0]:
def compute_acc_and_loss(model, criterion, loader):
    model.eval()
    loss_sum = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_sum = loss_sum + loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct*100/total, loss_sum/len(loader)

In [0]:
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device



train_loss = np.zeros(Epochs)
valid_loss = np.zeros(Epochs)

train_acc = np.zeros(Epochs)
valid_acc = np.zeros(Epochs)

for epoch in range(Epochs):  # loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = model(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.item()))
    pbar.set_description(' ')
    train_acc[epoch], train_loss[epoch] = compute_acc_and_loss(model, criterion, trainloader)
    valid_acc[epoch], valid_loss[epoch] = compute_acc_and_loss(model, criterion, validationloader)
    print('Train Epoch: {}  Train Accuracy: {:.2f} Train Loss : {:.6f} \
                            Validation Accuracy: {:.2f}  Validation Loss : {:.6f} '.format( epoch, 
                            train_acc[epoch], train_loss[epoch], 
                            valid_acc[epoch], valid_loss[epoch]))
    torch.save(model.state_dict(), out_dir + '/ft_' + str(epoch) +  '.pth')

print('Finished Training')


In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
np.save(output + 'task1_vgg16.npy', [train_loss, valid_loss, train_acc, valid_acc])

## Visualization

## Task1 (i): VGG16

In [0]:
model = models.vgg16(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=25088, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.classifier = nn.Sequential(*fc_layers)
for param in model.features.parameters():
    param.requires_grad = False
name = 'vgg16'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('model created')

In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
train_loss, valid_loss, train_acc, valid_acc  = np.load(output + 'task1_' + name + '.npy')
best_model = np.argmax(valid_acc)
r = best_model+1

plt.figure()
plt.plot(np.arange(r), train_loss[:r], 'r', np.arange(r), valid_loss[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0.15, 0.55)
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(np.arange(r), train_acc[:r], 'r', np.arange(r), valid_acc[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(70, 100)
plt.legend(['Training Acc', 'Validation Acc'])
plt.show()

print(best_model, valid_acc[best_model])

In [0]:
weights_path = '/content/drive/My Drive/DeepAssignment/Assignment05/models1' + '/vgg16_FC_Only' + '.pth'
model.load_state_dict(torch.load(weights_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('loaded')

In [0]:
print('Training Dataset: ')
print_statistics(model, trainloader)
print('Validation Dataset: ')
print_statistics(model, validationloader)
print('Test Dataset: ')
print_statistics(model, testloader)

## Task1 (ii): ResNet18

In [0]:
model = models.resnet18(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=512, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(*fc_layers)
name = 'resnet18'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task1/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('model created')

In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
train_loss, valid_loss, train_acc, valid_acc  = np.load(output + 'task1_' + name + '.npy')
best_model = np.argmax(valid_acc)
r = best_model+1

plt.figure()
plt.plot(np.arange(r), train_loss[:r], 'r', np.arange(r), valid_loss[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0.15, 0.55)
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(np.arange(r), train_acc[:r], 'r', np.arange(r), valid_acc[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(70, 100)
plt.legend(['Training Acc', 'Validation Acc'])
plt.show()

print(best_model, valid_acc[best_model])

In [0]:
weights_path = '/content/drive/My Drive/DeepAssignment/Assignment05/models1' + '/res18_FC_Only' + '.pth'
model.load_state_dict(torch.load(weights_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('loaded')

In [0]:
print('Training Dataset: ')
print_statistics(model, trainloader)
print('Validation Dataset: ')
print_statistics(model, validationloader)
print('Test Dataset: ')
print_statistics(model, testloader)

## Task2 (i)- a VGG16 (Conv1, Conv2, Conv3 are frozen)

In [0]:
model = models.vgg16(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=25088, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.classifier = nn.Sequential(*fc_layers)
for i,param in enumerate(model.features[:17].parameters()): # freezing 7 conv layers, and remaining 6 are unfrozen
    param.requires_grad = False
name = 'vgg16'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2a/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('model created')

In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
train_loss, valid_loss, train_acc, valid_acc  = np.load(output + 'task2a_' + name + '.npy')
best_model = np.argmax(valid_acc)
r = best_model+1

plt.figure()
plt.plot(np.arange(r), train_loss[:r], 'r', np.arange(r), valid_loss[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0.15, 0.55)
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(np.arange(r), train_acc[:r], 'r', np.arange(r), valid_acc[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(70, 100)
plt.legend(['Training Acc', 'Validation Acc'])
plt.show()

print(best_model, valid_acc[best_model])

In [0]:
weights_path = '/content/drive/My Drive/DeepAssignment/Assignment05/models1' + '/vgg16_c45' + '.pth'
model.load_state_dict(torch.load(weights_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('loaded')

In [0]:
print('Training Dataset: ')
print_statistics(model, trainloader)
print('Validation Dataset: ')
print_statistics(model, validationloader)
print('Test Dataset: ')
print_statistics(model, testloader)

## Task2 (i)- b VGG16 (Conv1, Conv2 are frozen)

In [0]:
model = models.vgg16(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=25088, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.classifier = nn.Sequential(*fc_layers)
for param in model.features[:10].parameters(): # freezing 4 conv layers, and remaining 9 are unfrozen
    param.requires_grad = False
name = 'vgg16'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2b/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('model created')

In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
train_loss, valid_loss, train_acc, valid_acc  = np.load(output + 'task2b_' + name + '.npy')
best_model = np.argmax(valid_acc)
r = best_model+1

plt.figure()
plt.plot(np.arange(r), train_loss[:r], 'r', np.arange(r), valid_loss[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0.15, 0.55)
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(np.arange(r), train_acc[:r], 'r', np.arange(r), valid_acc[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(70, 100)
plt.legend(['Training Acc', 'Validation Acc'])
plt.show()

print(best_model, valid_acc[best_model])

In [0]:
weights_path = '/content/drive/My Drive/DeepAssignment/Assignment05/models1' + '/vgg16_c345' + '.pth'
model.load_state_dict(torch.load(weights_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('loaded')

In [0]:
print('Training Dataset: ')
print_statistics(model, trainloader)
print('Validation Dataset: ')
print_statistics(model, validationloader)
print('Test Dataset: ')
print_statistics(model, testloader)

## Task2 (i)- c VGG16 (nothing is frozen)

In [0]:
model = models.vgg16(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=25088, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.classifier = nn.Sequential(*fc_layers)
name = 'vgg16'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2c/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('model created')

In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
train_loss, valid_loss, train_acc, valid_acc  = np.load(output + 'task2c_' + name + '.npy')
best_model = np.argmax(valid_acc)
r = best_model+1

plt.figure()
plt.plot(np.arange(r), train_loss[:r], 'r', np.arange(r), valid_loss[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0.15, 0.55)
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(np.arange(r), train_acc[:r], 'r', np.arange(r), valid_acc[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(70, 100)
plt.legend(['Training Acc', 'Validation Acc'])
plt.show()

print(best_model, valid_acc[best_model])

In [0]:
weights_path = '/content/drive/My Drive/DeepAssignment/Assignment05/models1' + '/vgg16_entire' + '.pth'
model.load_state_dict(torch.load(weights_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('loaded')

In [0]:
print('Training Dataset: ')
print_statistics(model, trainloader)
print('Validation Dataset: ')
print_statistics(model, validationloader)
print('Test Dataset: ')
print_statistics(model, testloader)

## Task2 (ii)- a resnet18 (Layer1, Layer2, Layer3 are frozen)

In [0]:
model = models.resnet18(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=512, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.fc = nn.Sequential(*fc_layers)
for i, child in enumerate(model.children()):
    if i < 7:
        for param in child.parameters():
            param.requires_grad = False
name = 'resnet18'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2a/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('model created')

In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
train_loss, valid_loss, train_acc, valid_acc  = np.load(output + 'task2a_' + name + '.npy')
best_model = np.argmax(valid_acc)
r = best_model+1

plt.figure()
plt.plot(np.arange(r), train_loss[:r], 'r', np.arange(r), valid_loss[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0.15, 0.55)
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(np.arange(r), train_acc[:r], 'r', np.arange(r), valid_acc[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(70, 100)
plt.legend(['Training Acc', 'Validation Acc'])
plt.show()

print(best_model, valid_acc[best_model])

In [0]:
weights_path = '/content/drive/My Drive/DeepAssignment/Assignment05/models1' + '/res18_L4' + '.pth'
model.load_state_dict(torch.load(weights_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('loaded')

In [0]:
print('Training Dataset: ')
print_statistics(model, trainloader)
print('Validation Dataset: ')
print_statistics(model, validationloader)
print('Test Dataset: ')
print_statistics(model, testloader)

## Task2 (ii)- b resnet18 (layer1 is frozen)

In [0]:
model = models.resnet18(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=512, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.fc = nn.Sequential(*fc_layers)
for i, child in enumerate(model.children()):
    if i < 5:
        for param in child.parameters():
            param.requires_grad = False
name = 'resnet18'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2b/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('model created')

In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
train_loss, valid_loss, train_acc, valid_acc  = np.load(output + 'task2b_' + name + '.npy')
best_model = np.argmax(valid_acc)
r = best_model+1

plt.figure()
plt.plot(np.arange(r), train_loss[:r], 'r', np.arange(r), valid_loss[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0.15, 0.55)
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(np.arange(r), train_acc[:r], 'r', np.arange(r), valid_acc[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(70, 100)
plt.legend(['Training Acc', 'Validation Acc'])
plt.show()

print(best_model, valid_acc[best_model])

In [0]:
weights_path = '/content/drive/My Drive/DeepAssignment/Assignment05/models1' + '/res18_L234' + '.pth'
model.load_state_dict(torch.load(weights_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('loaded')

In [0]:
print('Training Dataset: ')
print_statistics(model, trainloader)
print('Validation Dataset: ')
print_statistics(model, validationloader)
print('Test Dataset: ')
print_statistics(model, testloader)

## Task2 (ii)- c (nothing is frozen)

In [0]:
model = models.resnet18(pretrained=True)
fc_layers = []
fc_layers.extend([nn.Linear(in_features=512, out_features=130)])
fc_layers.extend([nn.ReLU(inplace=True)])
fc_layers.extend([nn.Linear(in_features=130, out_features=2)])
model.fc = nn.Sequential(*fc_layers)
name = 'resnet18'
out_dir = '/content/drive/My Drive/DeepAssignment/Assignment05/models/task2c/' + name
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('model created')

In [0]:
output = '/content/drive/My Drive/DeepAssignment/Assignment05/models/'
train_loss, valid_loss, train_acc, valid_acc  = np.load(output + 'task2c_' + name + '.npy')
best_model = np.argmax(valid_acc)
r = best_model+1

plt.figure()
plt.plot(np.arange(r), train_loss[:r], 'r', np.arange(r), valid_loss[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0.15, 0.55)
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(np.arange(r), train_acc[:r], 'r', np.arange(r), valid_acc[:r], 'b')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(70, 100)
plt.legend(['Training Acc', 'Validation Acc'])
plt.show()

print(best_model, valid_acc[best_model])

In [0]:
weights_path = '/content/drive/My Drive/DeepAssignment/Assignment05/models1' + '/res18_entire' + '.pth'
model.load_state_dict(torch.load(weights_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print('loaded')

In [0]:
print('Training Dataset: ')
print_statistics(model, trainloader)
print('Validation Dataset: ')
print_statistics(model, validationloader)
print('Test Dataset: ')
print_statistics(model, testloader)

## Helper

In [0]:
def print_statistics(model, loader):
    model.eval()
    actual_labels = []
    predic_labels = []

    cm = np.zeros((2,2))

    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))

    correct = 0
    total = 0


    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
        
            actual_labels.extend(labels)
            predic_labels.extend(predicted)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            c = (predicted == labels).squeeze()
            for i in np.arange(len(labels)):
                cm[labels[i], predicted[i]] = cm[labels[i], predicted[i]] + 1
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(2):
        print('  Accuracy of %5s : %.2f %%' % (class_names[i], 100 * class_correct[i] / class_total[i]))
    print('  Accuracy on whole test set : %.2f %%' %(100*correct/total) )
    print('  F1-Score on whole test set : %.2f %%' %(f1_score(torch.FloatTensor(actual_labels), torch.FloatTensor(predic_labels))))

    df_cm = pd.DataFrame(cm.astype(int), columns=class_names, index = class_names)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (4,4))
    sn.heatmap(df_cm, annot=True,annot_kws={"size": 10}, fmt="d")
    plt.show()